# Attendance System – Model Training

This notebook trains an emotion-detection CNN (FER2013) and builds a face-recognition embedding database for the automated attendance project.

In [2]:
# Section 1 – Imports & Environment Check
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import cv2
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mtcnn import MTCNN
import face_recognition
from tqdm import tqdm

print('TensorFlow version:', tf.__version__)
print('GPU Available:', tf.config.list_physical_devices('GPU'))

c:\Users\pawar\attendance_env\lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


TensorFlow version: 2.15.0
GPU Available: []


In [3]:
# Section 2 – Load & Pre-process FER2013 Dataset
def load_fer2013_data(data_path='fer2013_images'):
    """Load and preprocess FER2013 dataset"""
    if not os.path.exists(data_path):
        print(f"Dataset not found at {data_path}")
        print("Please download FER2013 dataset from Kaggle")
        return None, None, None, None

    # Read the dataset
    df = pd.read_csv(data_path)
    print(f"Dataset shape: {df.shape}")
    print(f"Emotion distribution:\n{df['emotion'].value_counts()}")

    # Emotion labels
    emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

    # Convert pixel strings to numpy arrays
    def pixels_to_array(pixel_string):
        return np.array(pixel_string.split(), dtype=np.float32).reshape(48, 48, 1)

    # Process images
    X = np.array([pixels_to_array(pixels) for pixels in tqdm(df['pixels'], desc='Processing images')])
    y = df['emotion'].values

    # Normalize pixel values
    X = X / 255.0

    # Convert labels to categorical
    y = keras.utils.to_categorical(y, 7)

    # Split data (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

# Load the FER2013 data
X_train, X_test, y_train, y_test = load_fer2013_data()

Dataset not found at fer2013_images
Please download FER2013 dataset from Kaggle


In [4]:
# Section 3 – Emotion Detection Model Architecture
def create_emotion_model():
    """Create CNN model for emotion detection"""
    model = keras.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Third Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Fully Connected Layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(7, activation='softmax')  # 7 emotions
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Create the emotion detection model
emotion_model = create_emotion_model()
emotion_model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 46, 46, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 22, 22, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 22, 22, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 64)      

In [5]:
# Section 4 – Train Emotion Detection Model
if X_train is not None:
    # Training callbacks
    callbacks = [
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5),
        keras.callbacks.ModelCheckpoint('models/emotion_model_best.h5', save_best_only=True)
    ]

    # Create models directory if it doesn't exist
    os.makedirs('models', exist_ok=True)

    # Train the model
    history = emotion_model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=50,
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        verbose=1
    )

    # Save the final model
    emotion_model.save('models/emotion_model.h5')
    print(" Emotion model saved successfully")
else:
    print(" Skipping training - dataset not available")

 Skipping training - dataset not available


In [6]:
# Section 5 – Model Evaluation
if X_train is not None:
    # Evaluate the model
    test_loss, test_accuracy = emotion_model.evaluate(X_test, y_test, verbose=0)
    print(f'Test accuracy: {test_accuracy:.4f}')

    # Plot training history
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig('models/training_history.png')
    plt.show()

In [7]:
# Section 6 – Face Recognition Setup
def create_face_embeddings_database(student_images_path='data/students'):
    """Create face embeddings database for student recognition"""
    face_embeddings = {}
    student_info = {}

    if not os.path.exists(student_images_path):
        print(f"Student images directory not found: {student_images_path}")
        print("Please create the directory and add student photos")
        return face_embeddings, student_info

    # Process each student directory
    for student_dir in os.listdir(student_images_path):
        student_path = os.path.join(student_images_path, student_dir)
        if not os.path.isdir(student_path):
            continue

        print(f"Processing student: {student_dir}")
        student_embeddings = []

        # Process each image in student directory
        for img_file in os.listdir(student_path):
            if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(student_path, img_file)

                # Load and process image
                image = face_recognition.load_image_file(img_path)
                face_encodings = face_recognition.face_encodings(image)

                if face_encodings:
                    student_embeddings.append(face_encodings[0])
                    print(f" Processed: {img_file}")
                else:
                    print(f"  No face found in: {img_file}")

        if student_embeddings:
            # Store average embedding for the student
            avg_embedding = np.mean(student_embeddings, axis=0)
            face_embeddings[student_dir] = avg_embedding

            # Extract student info from directory name (format: ID_Name)
            if '_' in student_dir:
                student_id, student_name = student_dir.split('_', 1)
                student_info[student_dir] = {
                    'id': student_id,
                    'name': student_name.replace('_', ' ')
                }
            else:
                student_info[student_dir] = {
                    'id': student_dir,
                    'name': student_dir
                }

    # Save embeddings
    os.makedirs('models', exist_ok=True)
    with open('models/face_embeddings.pkl', 'wb') as f:
        pickle.dump(face_embeddings, f)

    with open('models/student_info.pkl', 'wb') as f:
        pickle.dump(student_info, f)

    print(f" Face embeddings created for {len(face_embeddings)} students")
    return face_embeddings, student_info

# Create the face embeddings database
face_embeddings, student_info = create_face_embeddings_database()

Student images directory not found: data/students
Please create the directory and add student photos


In [8]:
# Section 7 – Test Face Detection Pipeline
detector = MTCNN()

def test_face_detection(image_path):
    """Test face detection on a sample image"""
    if not os.path.exists(image_path):
        print(f"Test image not found: {image_path}")
        return

    # Load image
    image = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect faces
    faces = detector.detect_faces(rgb_image)

    print(f"Detected {len(faces)} face(s)")

    # Draw bounding boxes
    for face in faces:
        x, y, width, height = face['box']
        cv2.rectangle(image, (x, y), (x + width, y + height), (255, 0, 0), 2)

        # Add confidence text
        confidence = face['confidence']
        cv2.putText(image, f'{confidence:.2f}', (x, y-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display result
    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Face Detection Test')
    plt.axis('off')
    plt.show()

# Uncomment to test with a sample image:
# test_face_detection('data/test_image.jpg')

In [9]:
# Section 8 – Model Export Summary
print("\n=== MODEL TRAINING SUMMARY ===")
print(" Files created:")
print(" - models/emotion_model.h5 (Emotion detection model)")
print(" - models/emotion_model_best.h5 (Best checkpoint)")
print(" - models/face_embeddings.pkl (Face recognition database)")
print(" - models/student_info.pkl (Student information)")
print(" - models/training_history.png (Training plots)\n")
print(" Next steps:")
print(" 1. Add student photos to data/students/ directory")
print(" 2. Run the main attendance system")
print(" 3. Configure time window (9:30 AM - 10:00 AM) in config.py\n")
print(" Note: If model files are large (>100MB), upload to Google Drive")


=== MODEL TRAINING SUMMARY ===
 Files created:
 - models/emotion_model.h5 (Emotion detection model)
 - models/emotion_model_best.h5 (Best checkpoint)
 - models/face_embeddings.pkl (Face recognition database)
 - models/student_info.pkl (Student information)
 - models/training_history.png (Training plots)

 Next steps:
 1. Add student photos to data/students/ directory
 2. Run the main attendance system
 3. Configure time window (9:30 AM - 10:00 AM) in config.py

 Note: If model files are large (>100MB), upload to Google Drive
